In [1]:
!pip install confluent_kafka sseclient

In [10]:
from confluent_kafka import SerializingProducer, DeserializingConsumer
from confluent_kafka.serialization import StringSerializer, StringDeserializer
from confluent_kafka.admin import AdminClient, NewTopic
from uuid import uuid4
import sys, random


brokers = "kafka1:9092,kafka2:9093"
topic = "taxi_rides"

In [11]:
pconf = {
    'bootstrap.servers': brokers,
    'partitioner': 'murmur2_random',
    'key.serializer': StringSerializer('utf_8'),
    'value.serializer':  StringSerializer('utf_8')
}

In [12]:
p = SerializingProducer(pconf)

In [13]:
from datetime import datetime  
  #2023-10-13T08:16:13Z
def construct_stock(row):
    time_stamp = time.time()
    date_time = datetime.fromtimestamp(time_stamp)
    str_date_time = date_time.strftime("%Y-%m-%dT%H:%M:%SZ") #"%d-%m-%Y, %H:%M:%S"
    stock = {"TODO": row[6],
             "TODO": float(row[2]),
             "timestamp":str_date_time
             }
    return stock


def construct_taxi_ride(row):
    time_stamp = time.time()
    date_time = datetime.fromtimestamp(time_stamp)
    str_date_time = date_time.strftime("%Y-%m-%dT%H:%M:%SZ")  # ISO format timestamp
    taxi_ride = {
        "medallion": row[0],
        "hack_license": row[1],
        "pickup_datetime": row[5],
        "dropoff_datetime": row[6],
        "pickup_longitude": float(row[10]),
        "pickup_latitude": float(row[11]),
        "dropoff_longitude": float(row[12]),
        "dropoff_latitude": float(row[13]),
        "timestamp": str_date_time
    }
    return taxi_ride

In [16]:
import csv, json
import time

n = 0
delay_in_seconds = 3
print_nth = 100

with open('sample.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    header = next(csv_reader)  # Skip the header row
    try:
        for row in csv_reader:
            #if n == 100:
             #  break
            taxi_ride = construct_taxi_ride(row)
            # Uncomment this to see every nth sent row
            if n % print_nth == 0:
                print()
                print(taxi_ride)
            else:
                print(".", end="")
            p.produce(topic, value=json.dumps(taxi_ride))
            p.poll(0)
            # Commented this out since it creates a 5 minute delay between sending each row
            #p.flush()
            time.sleep(delay_in_seconds)
            n += 1
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))

    


{'medallion': '89D227B655E5C82AECF13C3F540D4CF4', 'hack_license': 'BA96DE419E711691B9445D6A6307C170', 'pickup_datetime': '2013-01-01 15:11:48', 'dropoff_datetime': '2013-01-01 15:18:10', 'pickup_longitude': -73.978165, 'pickup_latitude': 40.757977, 'dropoff_longitude': -73.989838, 'dropoff_latitude': 40.751171, 'timestamp': '2024-06-04T13:57:37Z'}
....................................

KeyboardInterrupt: 

### Use only if you need to purge all the messages in the queue

In [ ]:
brokers = "kafka1:9092,kafka2:9093"

admin_client = AdminClient({"bootstrap.servers":brokers})
admin_client.delete_topics(topics=["taxi_rides"])